In [4]:
## create ground truth label from batch results
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join


In [7]:
## Load the CSV files and save in 1 dataframe
csv_files = [f for f in listdir("batch_results") if isfile(join("batch_results", f))]

# all_results = pd.DataFrame(columns=["anchor_id", "anchor_label", image_id", "melanoma"])
all_results = pd.DataFrame()

for i in csv_files:
    temp_df = pd.read_csv("batch_results/"+i)
    all_results = all_results.append(temp_df, ignore_index=True)
    
df_prep0 = all_results

#print(all_results.columns)


#print(df_prep0)

## Create the ground truth file
# Step 1: give image_id to picture that has been answered

df_prep1 = df_prep0
## Find image ID based on the answer is A or N
df_prep1[["image_id"]] = df_prep1[["image2"]].where(df_prep1[["answer"]].values=="A")
## Because it is binary, we now all NaN is B (Image 3)
df_prep1.image_id.fillna(df_prep1.image3,inplace=True)
print(df_prep1)

#See number of each id's in the list.. we want an average of >3
count_check = df_prep1['image_id'].value_counts()
df_prep1c = df_prep1.join(count_check, on="image_id", how = "left", rsuffix ="_count")
print(df_prep1c)
# Drop all columns with a value lower than 3
df_prep1d = df_prep1c[df_prep1c.image_id_count >= 1]
print(df_prep1d)



# Step 2: define the label equal to the anchor
df_prep2 = pd.DataFrame()
df_prep2[["image_id", "melanoma"]] = df_prep1d[["image_id", "anchor_label"]]
df_prep2 = df_prep2.sort_values("image_id")
print(df_prep2)

# Step 3: Take average of all answers 
df_prep3 = df_prep2
df_prep3[["melanoma"]] = df_prep3["melanoma"].astype(float)
df_prep3 = df_prep3.groupby("image_id").mean()
print(df_prep3)

# Step 4: Round to nearest label {0,1}
df = df_prep3
df = df.round({'melanoma': 0})
print(df)


    Unnamed: 0        WorkerId                    SubmitTime  \
0            0  A173XBIUVI8WAW  Mon Nov 22 03:53:56 PST 2021   
1            6  A1F1QHAACQSB9E  Mon Nov 22 03:53:55 PST 2021   
2           16  A2RINH8P33GL6H  Mon Nov 22 05:21:58 PST 2021   
3           26   A3HKZTDVORNEZ  Mon Nov 22 03:56:30 PST 2021   
4           27  A3J9WEQVUIML4P  Mon Nov 22 03:54:58 PST 2021   
5           28  A3OZ8KF0HWSVWK  Mon Nov 22 03:55:30 PST 2021   
6           29  A3PBA2ZHVZFGA3  Mon Nov 22 03:54:01 PST 2021   
7           33   A4JKUHL6IT5FZ  Mon Nov 22 03:56:13 PST 2021   
8           36   ALS4ARR5D4BEC  Mon Nov 22 03:55:45 PST 2021   
9           42   AZ0F8SS8884VC  Mon Nov 22 03:54:23 PST 2021   
10          43   AZ0KS5UTO3EJ2  Mon Nov 22 03:54:45 PST 2021   
11           0   ATKLWAX72HM7A  Mon Nov 22 03:53:21 PST 2021   
12           1  A1TW2BZRRS874Z  Mon Nov 22 03:53:22 PST 2021   
13           2   ATKLWAX72HM7A  Mon Nov 22 03:53:31 PST 2021   
14           3  A1TW2BZRRS874Z  Mon Nov 

In [11]:
df_prep1c[["image_id", "image_id_count"]].to_csv("Valid_Annotations_Batch.csv")